In [1]:
import os

In [2]:
%pwd

'd:\\ML Projects\\bird_type_classification_project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\ML Projects\\bird_type_classification_project'

In [5]:
import tensorflow as tf
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    test_data:Path
    all_params: dict
    params_image_size: list
    params_batch_size: int
    params_acceptable_classes:list

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model=Path("artifacts/training/model.h5"),
            training_data=Path("artifacts/data_ingestion/train"),
            test_data = Path("artifacts/data_ingestion/test"),
            all_params=self.params,
            params_acceptable_classes=self.params.ACCEPTABLE_CLASSES,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [8]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from cnnClassifier.utils.common import generate_dataframe
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        test_df = generate_dataframe(
            self.config.test_data, self.config.params_acceptable_classes
        )

        generator_kwargs = dict(
            rescale = 1./255,
        )

        dataflow_kwargs = dict(
            x_col = 'filepath',
            y_col = 'label',
            target_size = self.config.params_image_size[:-1],
            color_mode = 'rgb',
            batch_size = self.config.params_batch_size
        )

        ts_gen = ImageDataGenerator(
            **generator_kwargs
        )

        test_gen = ts_gen.flow_from_dataframe(
            dataframe=test_df,
            shuffle=True,
            **dataflow_kwargs
        )

        return test_gen

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        model = self.load_model(self.config.path_of_model)
        test_gen = self._valid_generator()
        score = model.evaluate(test_gen)
        return score

    
    def save_score(self, score):
        scores = {"loss": score[0], "accuracy": score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [9]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    score = evaluation.evaluation()
    evaluation.save_score(score)

except Exception as e:
   raise e

[20-Nov-23 01:53:56: INFO: common: config\config.yaml loaded successfully.]
[20-Nov-23 01:53:56: INFO: common: params.yaml loaded successfully.]
[20-Nov-23 01:53:56: INFO: common: Created directory at: artifacts]
Found 25 validated image filenames belonging to 5 classes.
1/1 [==============================] - 3s 3s/step - loss: 7.8619 - accuracy: 0.2000
[20-Nov-23 01:54:02: INFO: common: json file saved at: scores.json]
